# Protein Conformational Transitions calculations tutorial using BioExcel Building Blocks (biobb) and GOdMD

***
This tutorial aims to illustrate the process of computing a **conformational transition** between two known **structural conformations** of a protein, step by step, using the **BioExcel Building Blocks library (biobb)**. 

Examples shown are the calculation of XXX. 

The code wrapped is the ***GOdMD*** code:

**Exploration of conformational transition pathways from coarse-grained simulations.**<br>
*Sfriso P, Hospital A, Emperador A, Orozco M.*<br>
*Bioinformatics, 129(16):1980-6.*<br>
*Available at: https://doi.org/10.1093/bioinformatics/btt324*

***

## Settings

### Biobb modules used

 - [biobb_godmd](https://github.com/bioexcel/biobb_godmd): Tools to compute protein conformational transitions using GOdMD.
 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](https://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [plotly](https://plotly.com/python/): Python Open Source Graphing Library. 
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_godmd.git
cd biobb_wf_godmd
conda env create -f conda_env/environment.yml
conda activate biobb_wf_godmd
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_godmd/notebooks/biobb_wf_godmd.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Residue Mapping](#mapping)
 3. [Conformational Transition](#godmd)
 4. [Questions & Comments](#questions)
 
***

<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo" width="400" >

***

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **Auxiliar libraries**: Libraries to be used in the workflow are imported once at the beginning
 
 
 - **CP2K_DATA_DIR (TO REMOVE)**: Path to the CP2K parameter data files (BASIS_SET, POTENTIALS, etc.)

In [1]:
# TO BE REMOVED!!!

%load_ext autoreload
%autoreload 2

In [2]:
import os
import plotly
import plotly.graph_objs as go
import nglview
import ipywidgets

pdbOrigin = "1ake" # Chain A
pdbTarget = "4ake" # Chain A

In [3]:
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
originPDB = pdbOrigin+'.pdb'
targetPDB = pdbTarget+'.pdb'

prop_origin = {
    'pdb_code': pdbOrigin,
    'filter': False
}

prop_target = {
    'pdb_code': pdbTarget,
    'filter': False
}

# Launch bb for Origin PDB
pdb(output_pdb_path=originPDB,
    properties=prop_origin)

# Launch bb for Target PDB
pdb(output_pdb_path=targetPDB,
    properties=prop_target)

2022-09-20 17:15:59,239 [MainThread  ] [INFO ]  Executing biobb_io.api.pdb Version: 3.8.0
2022-09-20 17:15:59,240 [MainThread  ] [INFO ]  Downloading 1ake from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb1ake.ent
2022-09-20 17:15:59,741 [MainThread  ] [INFO ]  Writting pdb to: 1ake.pdb
2022-09-20 17:15:59,764 [MainThread  ] [INFO ]  Executing biobb_io.api.pdb Version: 3.8.0
2022-09-20 17:15:59,765 [MainThread  ] [INFO ]  Downloading 4ake from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb4ake.ent
2022-09-20 17:16:00,203 [MainThread  ] [INFO ]  Writting pdb to: 4ake.pdb


0

In [4]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(originPDB)
#view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['400px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(targetPDB)
#view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['400px','400px'])
view2.camera='orthographic'
view2
ipywidgets.HBox([view1, view2])

In [5]:
from biobb_structure_utils.utils.extract_chain import extract_chain

originPDB_chain = pdbOrigin + ".chains.pdb"
targetPDB_chain = pdbTarget + ".chains.pdb"

prop = {
    'chains': [ 'A' ]
}

# Launch bb for Origin PDB
extract_chain(
    input_structure_path=originPDB,
    output_structure_path=originPDB_chain,
    properties=prop
)

# Launch bb for Target PDB
extract_chain(
    input_structure_path=targetPDB,
    output_structure_path=targetPDB_chain,
    properties=prop
)

2022-09-20 17:16:02,488 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.extract_chain Version: 3.8.0
2022-09-20 17:16:02,489 [MainThread  ] [INFO ]  Selected Chains: A
2022-09-20 17:16:02,489 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_godmd/biobb_wf_godmd/notebooks/1ake.pdb -o 1ake.chains.pdb --force_save chains --select A

2022-09-20 17:16:04,295 [MainThread  ] [INFO ]  Exit code 0

2022-09-20 17:16:04,296 [MainThread  ] [INFO ]  ================================================================================
=                   BioBB structure checking utility v3.10.1                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22            =

Structure /Users/hospital/BioBB/Notebooks_dev/biobb_wf_godmd/biobb_wf_godmd/notebooks/1ake.pdb loaded
 Title: structure of the complex between adenylate kinase from escherichia coli and the inhibitor ap5a refined at 1.9 angstroms resolution: a model for a cataly

0

In [6]:
from biobb_structure_utils.utils.remove_molecules import remove_molecules

originPDB_chain_nolig = pdbOrigin + ".chains.nolig.pdb"

prop = {
    'molecules': [
        {
            'name' : 'AP5'
        }
    ]
}
remove_molecules(input_structure_path=originPDB_chain,
                 output_molecules_path=originPDB_chain_nolig,
                 properties=prop)


2022-09-20 17:16:04,778 [MainThread  ] [INFO ]  Executing biobb_structure_utils.utils.remove_molecules Version: 3.8.0
2022-09-20 17:16:04,833 [MainThread  ] [INFO ]  Writting pdb to: 1ake.chains.nolig.pdb
2022-09-20 17:16:04,834 [MainThread  ] [INFO ]  Removed: []


0

In [7]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(originPDB_chain_nolig)
#view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['400px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(targetPDB_chain)
#view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['400px','400px'])
view2.camera='orthographic'
view2
ipywidgets.HBox([view1, view2])

<a id="mapping"></a>
## Preparing
Preparing the **structures** and the **mapping** needed
 

In [8]:
from biobb_godmd.godmd.godmd_prep import godmd_prep

originALN = pdbOrigin + ".aln"
targetALN = pdbTarget + ".aln"

prop = {
    'gapopen' : '12.0',
    'gapextend' : '2',
    'remove_tmp': True
}

godmd_prep( input_pdb_orig_path=originPDB_chain_nolig,
            input_pdb_target_path=targetPDB_chain,
            output_aln_orig_path=originALN,
            output_aln_target_path=targetALN,
            properties=prop)

2022-09-20 17:16:07,705 [MainThread  ] [INFO ]  Creating 9ee41bdb-1bb5-4d39-a840-59ad3b96ba24 temporary folder
2022-09-20 17:16:07,707 [MainThread  ] [INFO ]  water -auto -outfile 9ee41bdb-1bb5-4d39-a840-59ad3b96ba24/water_align.out -asequence 9ee41bdb-1bb5-4d39-a840-59ad3b96ba24/1ake.chains.nolig.pdb.fa -bsequence 9ee41bdb-1bb5-4d39-a840-59ad3b96ba24/4ake.chains.pdb.fa -gapopen 12.0 -gapextend 2 -datafile EPAM250 -aformat markx10

2022-09-20 17:16:11,282 [MainThread  ] [INFO ]  Exit code 0

2022-09-20 17:16:11,285 [MainThread  ] [INFO ]  Removed: ['9ee41bdb-1bb5-4d39-a840-59ad3b96ba24']


0

<a id="godmd"></a>
## Running GOdMD 
Computing the **conformational transition** 
 

In [9]:
from biobb_godmd.godmd.godmd_run import godmd_run

godmd_log = pdbOrigin + "-" + pdbTarget + ".godmd.log"
godmd_trj = pdbOrigin + "-" + pdbTarget + ".godmd.mdcrd"
godmd_ene = pdbOrigin + "-" + pdbTarget + ".godmd.ene.out"

prop = {
    'remove_tmp': False,
    'godmdin':{
        'temp' : 400
    }
}

godmd_run(   input_pdb_orig_path=originPDB_chain_nolig,
             input_pdb_target_path=targetPDB_chain,
             input_aln_orig_path=originALN,
             input_aln_target_path=targetALN,
             output_log_path=godmd_log,
             output_ene_path=godmd_ene,
             output_trj_path=godmd_trj,
             properties=prop)


2022-09-20 17:16:11,994 [MainThread  ] [INFO ]  Creating 7859a500-ae98-4d05-aa17-8362b56cda29 temporary folder
2022-09-20 17:16:11,995 [MainThread  ] [INFO ]  discrete -i 7859a500-ae98-4d05-aa17-8362b56cda29/godmd.in -pdbin 1ake.chains.nolig.pdb -pdbtarg 4ake.chains.pdb -p1 1ake.aln -p2 4ake.aln -o 1ake-4ake.godmd.log -ener 1ake-4ake.godmd.ene.out -trj 1ake-4ake.godmd.mdcrd

2022-09-20 17:16:23,294 [MainThread  ] [INFO ]  Exit code 0



0

In [10]:
from biobb_analysis.ambertools.cpptraj_convert import cpptraj_convert

godmd_trj_dcd = pdbOrigin + "-" + pdbTarget + ".godmd.dcd"
reference_pdb = "reference.pdb"

prop = {
    'format': 'dcd'
}

cpptraj_convert(input_top_path=reference_pdb,
                input_traj_path=godmd_trj,
                output_cpptraj_path=godmd_trj_dcd,
                properties=prop)


2022-09-20 17:16:23,894 [MainThread  ] [INFO ]  Executing biobb_analysis.ambertools.cpptraj_convert Version: 3.8.0
2022-09-20 17:16:23,896 [MainThread  ] [INFO ]  cpptraj -i 6c723a1c-3452-40a4-ada1-b19e82738757/instructions.in

2022-09-20 17:16:27,585 [MainThread  ] [INFO ]  Exit code 0

2022-09-20 17:16:27,586 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V6.4.4 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 09/20/22 17:16:27
| Available memory: 2.717 GB

INPUT: Reading input from '6c723a1c-3452-40a4-ada1-b19e82738757/instructions.in'
  [parm reference.pdb]
	Reading 'reference.pdb' as PDB File
	Reading bond info from CONECT records.
	Not reading bond info from LINK records.
	Determining bond info from distances.
  [trajin 1ake-4ake.godmd.mdcrd 1 -1 1]
	Reading '1ake-4ake.godmd.mdcrd' as Amber Trajectory
  [strip !:*]
    STRIP: Stripping atoms in mask [!:*]
  [trajout 1ake-4ake.godmd.dcd dcd]
	Writing '1ake-4ake.godmd.dcd' as C

0

In [11]:
# Show trajectory

view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(godmd_trj_dcd, reference_pdb), gui=True)

view.add_representation(repr_type='tube', colorScheme = 'atomindex')

# Origin Structure
b = view.add_component(originPDB_chain_nolig)
b.clear_representations()
b.add_representation(repr_type='tube',
                        opacity=.4,
                        color='blue')

# Target Structure
c = view.add_component(targetPDB_chain)
c.clear_representations()
c.add_representation(repr_type='tube', 
                       opacity=.4,
                        color='red')

# Align origin and target
code = """
var stage = this.stage;
var clist_len = stage.compList.length;
var i = 0;
var s = [];
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined) {        
       s.push(stage.compList[i])
    }
}
NGL.superpose(s[1].structure, s[2].structure, true, ".CA")
s[ 1 ].updateRepresentations({ position: true })
s[ 1 ].autoView()
"""

view._execute_js_code(code)

view._remote_call('setSize', target='Widget', args=['800px','600px'])
view

NGLWidget(max_frame=99)

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
